<a href="https://colab.research.google.com/github/mladbago/podstawy_sztucznej_inteligencji/blob/main/Z11_A_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

print(keras.__version__)

2.12.0
2.12.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
# Wczytaj dane treningowe i testowe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train_set = pd.read_csv('https://raw.githubusercontent.com/mladbago/podstawy_sztucznej_inteligencji/main/Dane/adult/adult.data', sep=", ",header = None)
test_set = pd.read_csv('https://raw.githubusercontent.com/mladbago/podstawy_sztucznej_inteligencji/main/Dane/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()


<ipython-input-3-d866d9954fc4>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_set = pd.read_csv('https://raw.githubusercontent.com/mladbago/podstawy_sztucznej_inteligencji/main/Dane/adult/adult.data', sep=", ",header = None)
<ipython-input-3-d866d9954fc4>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_set = pd.read_csv('https://raw.githubusercontent.com/mladbago/podstawy_sztucznej_inteligencji/main/Dane/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Zad.
Zróbmy szybki preprocesing

In [4]:
dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class


# TensorBoard

In [5]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [6]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2023_05_31-17_40_29'

In [7]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
from keras.models import Sequential
from keras.layers import Dense

from keras.callbacks import History

history = History()
model = Sequential()
model.add(Dense(100,activation="sigmoid",input_shape=(X_train.shape[1],)))
model.add(Dense(50,activation="sigmoid"))
model.add(Dense(10,activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4200      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,771
Trainable params: 9,771
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss="binary_crossentropy",optimizer="Adam", metrics=["accuracy"])

In [10]:
from keras.callbacks import TensorBoard 

tensorboard_cb = TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), callbacks=[tensorboard_cb])

Epoch 1/30
943/943 [==============================] - 3s 3ms/step - loss: 0.4467 - accuracy: 0.7885 - val_loss: 0.3979 - val_accuracy: 0.8021
Epoch 2/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3798 - accuracy: 0.8108 - val_loss: 0.3738 - val_accuracy: 0.8113
Epoch 3/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3761 - accuracy: 0.8109 - val_loss: 0.3683 - val_accuracy: 0.8202
Epoch 4/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3742 - accuracy: 0.8121 - val_loss: 0.4026 - val_accuracy: 0.8092
Epoch 5/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3719 - accuracy: 0.8126 - val_loss: 0.3662 - val_accuracy: 0.8163
Epoch 6/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3703 - accuracy: 0.8120 - val_loss: 0.3723 - val_accuracy: 0.8075
Epoch 7/30
943/943 [==============================] - 2s 2ms/step - loss: 0.3706 - accuracy: 0.8148 - val_loss: 0.3671 - val_accuracy: 0.8186
Epoch 

o start the TensorBoard server, one option is to open a terminal, if needed activate the virtualenv where you installed TensorBoard, go to this notebook's directory, then type:

```python
$ tensorboard --logdir=./my_logs --port=6006
```

You can then open your web browser to localhost:6006 and use TensorBoard. Once you are done, press Ctrl-C in the terminal window, this will shutdown the TensorBoard server.

Alternatively, you can load TensorBoard's Jupyter extension and run it like this:

```python
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006
```

Gdyby nie działał tensorboard

https://github.com/pytorch/pytorch/issues/22676
```python
pip uninstall tb-nightly tensorboardX tensorboard
pip install tensorboard
```

# Zad

* Naucz dowolny model na zbiorze MNIST.
* Wyświetl krzywe uczenia w tensorboard

In [11]:
import datetime
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

11490434/11490434 [==============================] - 0s 0us/step


In [12]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2210 - accuracy: 0.9353 - val_loss: 0.1046 - val_accuracy: 0.9684
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0979 - accuracy: 0.9697 - val_loss: 0.0891 - val_accuracy: 0.9713
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0691 - accuracy: 0.9779 - val_loss: 0.0659 - val_accuracy: 0.9790
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0529 - accuracy: 0.9829 - val_loss: 0.0730 - val_accuracy: 0.9784
Epoch 5/5
1875/1875 [==============================] - 8s 5ms/step - loss: 0.0437 - accuracy: 0.9860 - val_loss: 0.0680 - val_accuracy: 0.9786
